In [0]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np

In [0]:
cd ./drive/My Drive/WBA/gls

/content/drive/My Drive/WBA/gls


In [0]:
path_very_good = "./lettuce/extended_lettuce/very_good2"
path_good = "./lettuce/extended_lettuce/good2"
path_bad = "./lettuce/extended_lettuce/bad2"

In [0]:
import os  #必ず必要
pic_list_very_good = os.listdir(path_very_good)
pic_list_good = os.listdir(path_good)
pic_list_bad = os.listdir(path_bad)

In [0]:
%tensorflow_version 1.x
#入力画像の読み込み
from tensorflow.python.keras.preprocessing.image import load_img
import numpy as np

#ロードした画像データを格納するからのリスト
# 猫の画像をロードして、リサイズ
  
img_list_very_good = [load_img(path_very_good+"/"+name,target_size=(224,224)) for name in pic_list_very_good]
img_list_good = [load_img(path_good+"/"+name,target_size=(224,224)) for name in pic_list_good]
img_list_bad = [load_img(path_bad+"/"+name,target_size=(224,224)) for name in pic_list_bad]

In [0]:
from tensorflow.python.keras.preprocessing.image import img_to_array
def img_to_arr(img_list):
    arr_list = [img_to_array(img) for img in img_list]
    arr_list = np.array(arr_list)
    arr_list = arr_list/255
    return arr_list
arr_list_very_good = img_to_arr(img_list_very_good)
arr_list_good = img_to_arr(img_list_good)
arr_list_bad = img_to_arr(img_list_bad)

In [0]:
print(arr_list_bad.shape)
print(arr_list_good.shape)
print(arr_list_very_good.shape)

In [0]:
from sklearn.model_selection import train_test_split
bad_train,bad_test= train_test_split(arr_list_bad)
good_train,good_test= train_test_split(arr_list_good)
very_good_train,very_good_test= train_test_split(arr_list_very_good)
print(bad_train.shape)
print(good_train.shape)
print(very_good_train.shape)

In [0]:
x_train = np.hstack(([bad_train],[good_train],[very_good_train]))
x_train = x_train[0]
print(x_train.shape)
x_test = np.hstack(([bad_test],[good_test],[very_good_test]))
x_test = x_test[0]
print(x_test.shape)

In [0]:
y_train=[[1,0,0] for i in range(bad_train.shape[0])]
for i in range(good_train.shape[0]):
    y_train.append([0,1,0])
for i in range(very_good_train.shape[0]):
    y_train.append([0,0,1])
y_train=np.array(y_train)
y_train.shape

In [0]:
y_test=[[1,0,0] for i in range(bad_test.shape[0])]
for i in range(good_test.shape[0]):
    y_test.append([0,1,0])
for i in range(very_good_test.shape[0]):
    y_test.append([0,0,1])
y_test=np.array(y_test)
y_test.shape

In [0]:
from imblearn.over_sampling import RandomOverSampler

In [0]:
from tensorflow.python.keras.applications.vgg16 import VGG16
vgg16 = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg16.summary()

In [0]:
model = None
model

In [0]:
# レイヤーの調整
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout,Flatten

def build_transfer_model(vgg16):
    model = Sequential(vgg16.layers)
    for layer in model.layers[:15]:
        layer.trainable = False

    model.add(Flatten())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3,activation='sigmoid'))

    return model

model = build_transfer_model(vgg16)



In [0]:
from tensorflow.python.keras.optimizers import SGD
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(lr=1e-4, momentum=0.9),
    metrics=['accuracy']
)
# モデルのサマリを確認
model.summary()

In [0]:
import math
from tensorflow.python.keras.callbacks import TensorBoard
batch_s = 32
tsb = TensorBoard(log_dir='./logs')
steps_per_epoch = math.ceil(x_train.shape[0] / batch_s)
history_model= model.fit(
  x_train,
  y_train,
  batch_size= batch_s,
  epochs=55,
  validation_split=0.2,
  callbacks=[tsb]
)

In [0]:
from sklearn.metrics import confusion_matrix
# classification_report(y_test, pre_y, target_names=["bad", "good","very_good"])
print(confusion_matrix(y_test.argmax(axis=1), pre_y))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
model.save('./lettuce/lettuce_model.h5', include_optimizer=False)

In [0]:
pre_y = model.predict(x_test).argmax(axis=1)
pre_y

In [0]:
pre_bad = model.predict(bad_test)
result_bad=pre_bad.argmax(axis=1)
result_bad

In [0]:
pre_good = model.predict(good_test)
result_good=pre_good.argmax(axis=1)
result_good

In [0]:
pre_very_good = model.predict(very_good_test)
result_very_good=pre_very_good.argmax(axis=1)
result_very_good

In [0]:
y_test

In [0]:
print(result_bad.shape,result_good.shape,result_very_good.shape)

In [0]:
img_list_very_good[0]